In [1]:
# %%
# Data Augmentation & Preprocessing
import sys
import os
import torch
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import imageio
import albumentations as A  # 确保安装: pip install albumentations

print(f"Torch version: {torch.__version__}")

""" 1. 定义预处理类 (特征堆叠的核心) """
class RetinaPreprocess:
    @staticmethod
    def get_green_enhanced_stack(img_rgb):
        """
        输入: RGB 图像 (H, W, 3)
        输出: 3通道特征图 [Green, CLAHE, TopHat]
        """
        # 1. 提取绿色通道
        g, b, r = cv2.split(img_rgb)
        green = g # OpenCV读入是BGR, 这里假设输入已经是RGB或者是BGR拆分正确
        # 如果输入是RGB: R,G,B = cv2.split(img); green = G
        # 为了保险，我们下面统一按 RGB 处理
        
        # 2. CLAHE (限制对比度自适应直方图均衡化)
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced_green = clahe.apply(green)
        
        # 3. Morphological TopHat (形态学顶帽变换 - 提取微血管)
        # 血管通常比背景暗，所以用 BlackHat (底帽) 或者反转后的 TopHat
        # 这里使用针对血管优化的变体
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        tophat = cv2.morphologyEx(enhanced_green, cv2.MORPH_TOPHAT, kernel)
        
        # 也可以尝试 BlackHat 提取暗细节:
        # blackhat = cv2.morphologyEx(enhanced_green, cv2.MORPH_BLACKHAT, kernel)
        
        # 堆叠成 3 通道: [原始Green, 增强Green, 形态学特征]
        # 归一化已经在 Dataset 里做了，这里保持 uint8
        merged = cv2.merge([green, enhanced_green, tophat])
        return merged

    @staticmethod
    def harmonize_od_color(image, od_mask):
        """
        简单的视盘区域亮度同化，减少视盘边缘的假阳性
        """
        if np.sum(od_mask) == 0:
            return image
        
        # 简单策略：将OD区域的亮度调整为邻域背景亮度 (这里简化为不做处理，防止引入伪影)
        # 如果没有完美的 OD Mask，建议跳过此步，避免引入硬边缘噪声
        return image

""" 2. 定义增强流水线 (Albumentations) """
def get_transforms(img_size=560, is_train=True):
    if is_train:
        return A.Compose([
            A.Resize(img_size, img_size),
            
            # 几何变换
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            
            # 仿射变换 (温和)
            A.Affine(
                scale=(0.95, 1.05),
                translate_percent=(0.02, 0.02),
                rotate=(-15, 15), # 适度旋转
                shear=(-5, 5),    # 适度剪切
                mode=cv2.BORDER_CONSTANT,
                cval=0,
                p=0.5
            ),
            
            # 甚至可以加入弹性变换模拟眼球曲率，但先保持简单
            
        ], additional_targets={'mask': 'mask'})
    else:
        # 测试集仅调整大小
        return A.Compose([
            A.Resize(img_size, img_size)
        ])

""" 3. Create Directory """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

''' 4. 加载原始数据路径 '''
def load_data(path):
    # DRIVE 数据集结构
    train_x = sorted(glob(os.path.join(path, "training", "images", "*.tif")))
    train_y = sorted(glob(os.path.join(path, "training", "1st_manual", "*.gif")))
    
    # 尝试加载 OD Mask (如果有的话，没有则忽略)
    # train_od = sorted(glob(os.path.join(path, "training", "mask", "*.gif"))) # 假设路径

    test_x = sorted(glob(os.path.join(path, "test", "images", "*.tif")))
    test_y = sorted(glob(os.path.join(path, "test", "1st_manual", "*.gif")))

    return (train_x, train_y), (test_x, test_y)

''' 5. 核心处理函数 '''
def process_and_augment(images, masks, save_path, augment_times=1, is_train=True):
    """
    augment_times: 每张图扩充多少倍 (仅对训练集有效)
    is_train: 是否为训练集 (决定是否应用几何增强)
    """
    transform = get_transforms(img_size=560, is_train=is_train)
    
    print(f"Processing {'Train' if is_train else 'Test'} data...")
    
    for idx, (x_path, y_path) in tqdm(enumerate(zip(images, masks)), total=len(images)):
        name = x_path.split(os.sep)[-1].split(".")[0]
        
        # 读取
        # OpenCV 读入是 BGR
        img_bgr = cv2.imread(x_path, cv2.IMREAD_COLOR) 
        mask = imageio.mimread(y_path)[0]

        if img_bgr is None or mask is None:
            continue

        # 转 RGB
        img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
        
        # === 核心步骤 1: 特征工程预处理 ===
        # 注意：无论是训练集还是测试集，都必须做这一步！
        # 这样网络学到的是 "Green+CLAHE+TopHat" 的特征，测试时也必须给它这个特征。
        feature_stack = RetinaPreprocess.get_green_enhanced_stack(img_rgb)
        
        # 准备增强循环
        # 如果是测试集，augment_times 强制为 1
        loops = augment_times if is_train else 1
        
        for i in range(loops):
            # === 核心步骤 2: 几何增强 (Albumentations) ===
            augmented = transform(image=feature_stack, mask=mask)
            
            aug_img = augmented['image']
            aug_mask = augmented['mask']
            
            # 命名
            if is_train:
                tmp_image_name = f"{name}_{i}.png"
                tmp_mask_name = f"{name}_{i}.png"
            else:
                # 测试集保持原名，方便对应
                tmp_image_name = f"{name}.png"
                tmp_mask_name = f"{name}.png"

            image_save_path = os.path.join(save_path, "image", tmp_image_name)
            mask_save_path = os.path.join(save_path, "mask", tmp_mask_name)

            # 保存
            # 注意：cv2.imwrite 期望 BGR 顺序。
            # 我们的 feature_stack 是 [Green, CLAHE, TopHat]。
            # 保存后，读取时 cv2.imread 也会按 [ch0, ch1, ch2] 读入，顺序一致，所以直接写即可。
            cv2.imwrite(image_save_path, aug_img)
            cv2.imwrite(mask_save_path, aug_mask)

if __name__ == "__main__":
    """ Seeding """
    np.random.seed(42)

    """ Load the data """
    data_path = "data/DRIVE/" # 请确保路径正确
    
    if os.path.exists(data_path):
        (train_x, train_y), (test_x, test_y) = load_data(data_path)

        print(f"Train: {len(train_x)} - {len(train_y)}")
        print(f"Test: {len(test_x)} - {len(test_y)}")

        """ Create directories """
        # 建议清理旧数据，防止混淆
        # import shutil
        # shutil.rmtree("working/new_data", ignore_errors=True)
        
        create_dir("working/new_data/train/image/")
        create_dir("working/new_data/train/mask/")
        create_dir("working/new_data/test/image/")
        create_dir("working/new_data/test/mask/")

        """ Run Processing """
        # 训练集：特征工程 + 10倍几何增强
        process_and_augment(train_x, train_y, "working/new_data/train/", augment_times=12, is_train=True)
        
        # 测试集：仅特征工程 + Resize (无几何增强)
        process_and_augment(test_x, test_y, "working/new_data/test/", augment_times=1, is_train=False)
        
        print("Data processing complete. The images now contain [Green, CLAHE, TopHat] channels.")
    else:
        print(f"Data path not found: {data_path}")

/home/wangziteng/miniconda3/envs/pampc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_1006381/3556601113.py:71: UserWarning: Argument(s) 'mode, cval' are not valid for transform Affine
  A.Affine(


Torch version: 2.5.1
Train: 20 - 20
Test: 20 - 20
Processing Train data...


100%|██████████| 20/20 [00:03<00:00,  6.58it/s]


Processing Test data...


100%|██████████| 20/20 [00:00<00:00, 54.31it/s]

Data processing complete. The images now contain [Green, CLAHE, TopHat] channels.


In [2]:
# Model (LFA-Net)
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalModulation(nn.Module):
    def __init__(self, in_channels, gamma=2.0, alpha=0.25):
        super(FocalModulation, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.gmp = nn.AdaptiveMaxPool2d(1)
        self.conv = nn.Conv2d(in_channels, in_channels, kernel_size=1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        mean = self.gap(x)
        max_val = self.gmp(x)
        modulation = (max_val - mean) * self.alpha
        modulation = self.conv(modulation)
        modulation = self.sigmoid(modulation)
        scaled_inputs = x * modulation
        outputs = torch.pow(scaled_inputs, self.gamma)
        return outputs

class FocalModulationContextAggregation(nn.Module):
    def __init__(self, in_channels, filters):
        super(FocalModulationContextAggregation, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, filters, kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        self.conv2 = nn.Conv2d(in_channels, filters, kernel_size=1)
        self.relu2 = nn.ReLU()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.conv_ctx = nn.Conv2d(filters, filters, kernel_size=1)
        self.sigmoid = nn.Sigmoid()
        self.focal_mod = FocalModulation(filters)

    def forward(self, x):
        c1 = self.relu1(self.conv1(x))
        c2 = self.relu2(self.conv2(x))
        
        global_context = self.gap(c2)
        global_context = self.sigmoid(self.conv_ctx(global_context))
        global_context = c1 * global_context
        
        fm = self.focal_mod(global_context)
        return torch.cat([c1, fm], dim=1)

class VisionMambaInspired(nn.Module):
    def __init__(self, dim, dropout_rate=0.1):
        super(VisionMambaInspired, self).__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.token_mixer = nn.Sequential(
            nn.Conv2d(dim, dim, kernel_size=3, padding=1, groups=dim),
            nn.GELU(),
            nn.Dropout(dropout_rate)
        )
        self.norm2 = nn.LayerNorm(dim)
        self.channel_mixer = nn.Sequential(
            nn.Linear(dim, dim * 4),
            nn.GELU(),
            nn.Dropout(dropout_rate),
            nn.Linear(dim * 4, dim),
            nn.Dropout(dropout_rate)
        )

    def forward(self, x):
        b, c, h, w = x.shape
        
        shortcut = x
        x_perm = x.permute(0, 2, 3, 1)
        x_norm = self.norm1(x_perm).permute(0, 3, 1, 2)
        x_tm = self.token_mixer(x_norm) + shortcut
        
        shortcut = x_tm
        x_perm = x_tm.permute(0, 2, 3, 1)
        x_norm = self.norm2(x_perm)
        x_cm = self.channel_mixer(x_norm)
        x_cm = x_cm.permute(0, 3, 1, 2)
        
        return x_cm + shortcut

class LiteFusionAttention(nn.Module):
    def __init__(self, in_channels, filters):
        super(LiteFusionAttention, self).__init__()
        self.proj1 = nn.Conv2d(in_channels, filters, kernel_size=1)
        self.norm = nn.LayerNorm(filters)
        self.conv = nn.Conv2d(filters, filters, kernel_size=3, padding=1)
        self.fmca = FocalModulationContextAggregation(filters, filters)
        self.proj2 = nn.Conv2d(2 * filters, filters, kernel_size=1)
        self.vm = VisionMambaInspired(filters)
        
        self.res_proj = nn.Conv2d(in_channels, filters, kernel_size=1) if in_channels != filters else nn.Identity()

    def forward(self, x):
        input_tensor = self.proj1(x)
        
        x_perm = input_tensor.permute(0, 2, 3, 1)
        x_norm = self.norm(x_perm).permute(0, 3, 1, 2)
        
        x_conv = self.conv(x_norm)
        x_fmca = self.fmca(x_conv)
        x_proj = self.proj2(x_fmca)
        
        res = self.res_proj(x) if isinstance(self.res_proj, nn.Conv2d) else input_tensor
        out = x_proj + res
        
        out = self.vm(out)
        return out

class RA_AttentionBlock(nn.Module):
    def __init__(self, in_channels, n_classes, k):
        super(RA_AttentionBlock, self).__init__()
        self.k = k
        self.n_classes = n_classes
        self.conv = nn.Conv2d(in_channels, k * n_classes, kernel_size=3, padding=1)
        self.bn = nn.BatchNorm2d(k * n_classes)
        self.relu = nn.ReLU()
        self.gap = nn.AdaptiveAvgPool2d(1)
        self.gmp = nn.AdaptiveMaxPool2d(1)

    def forward(self, x):
        b, c, h, w = x.shape
        f = self.relu(self.bn(self.conv(x)))
        
        x1 = self.gmp(f)
        x2 = self.gap(f)
        x_mul = x1 * x2
        
        x_reshape = x_mul.view(b, self.n_classes, self.k)
        s = torch.mean(x_reshape, dim=-1, keepdim=False)
        
        f_perm = f.permute(0, 2, 3, 1)
        f_reshape = f_perm.view(b, h, w, self.n_classes, self.k)
        f_mean = torch.mean(f_reshape, dim=-1, keepdim=False)
        
        s_expanded = s.view(b, 1, 1, self.n_classes)
        x_weighted = f_mean * s_expanded
        
        m = torch.mean(x_weighted, dim=-1, keepdim=True)
        m = m.permute(0, 3, 1, 2)
        
        semantic = x * m
        return semantic

class ConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, dropout_rate=0.5):
        super(ConvBlock, self).__init__()
        self.conv1x1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, padding=0, bias=False)
        self.conv3x3 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.conv3x3_dilated = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=2, dilation=2, bias=False)
        self.leaky_relu = nn.LeakyReLU(0.1)

    def forward(self, x):
        c1 = self.conv1x1(x)
        c3 = self.conv3x3(x)
        c3d = self.conv3x3_dilated(x)
        out = c1 + c3 + c3d
        out = self.leaky_relu(out)
        return out

class build_unet(nn.Module):
    def __init__(self, input_channels=3, num_classes=1, feature_scale=2, dropout=0.5):
        super(build_unet, self).__init__()
        filters = [int(x / feature_scale) for x in [16, 32, 64]]
        
        self.conv1 = ConvBlock(input_channels, filters[0], dropout)
        self.pool1 = nn.MaxPool2d(2)
        self.bn1 = nn.BatchNorm2d(filters[0])
        
        self.conv2 = ConvBlock(filters[0], filters[1], dropout)
        self.pool2 = nn.MaxPool2d(2)
        self.bn2 = nn.BatchNorm2d(filters[1])
        
        self.conv3 = ConvBlock(filters[1], filters[2], dropout)
        self.pool3 = nn.MaxPool2d(2)
        self.bn3 = nn.BatchNorm2d(filters[2])
        
        self.lfa = LiteFusionAttention(filters[2], filters=32)
        
        lfa_out_channels = 32 
        self.att1 = RA_AttentionBlock(lfa_out_channels, 1, 16)
        
        self.up1 = nn.ConvTranspose2d(lfa_out_channels * 2, filters[2], kernel_size=3, stride=2, padding=1, output_padding=1)
        
        self.att2 = RA_AttentionBlock(filters[1], 1, 16)
        self.dec_conv1 = nn.Conv2d(filters[1] + filters[2], filters[2], kernel_size=3, padding=1)
        self.relu1 = nn.ReLU()
        
        self.up2 = nn.ConvTranspose2d(filters[2], filters[2], kernel_size=3, stride=2, padding=1, output_padding=1)
        
        self.att3 = RA_AttentionBlock(filters[0], 1, 16)
        self.dec_conv2 = nn.Conv2d(filters[0] + filters[2], filters[2], kernel_size=3, padding=1)
        self.relu2 = nn.ReLU()
        
        self.up3 = nn.ConvTranspose2d(filters[2], filters[0], kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dec_conv3 = nn.Conv2d(filters[0], filters[0], kernel_size=3, padding=1)
        self.relu3 = nn.ReLU()
        
        self.final = nn.Conv2d(filters[0], num_classes, kernel_size=1)

    def forward(self, x):
        c1 = self.conv1(x)
        p1 = self.bn1(self.pool1(c1))
        
        c2 = self.conv2(p1)
        p2 = self.bn2(self.pool2(c2))
        
        c3 = self.conv3(p2)
        p3 = self.bn3(self.pool3(c3))
        
        lfa = self.lfa(p3)
        
        att1 = self.att1(lfa)
        fused = torch.cat([att1, lfa], dim=1)
        
        d1 = self.up1(fused)
        if d1.size() != c2.size():
             d1 = F.interpolate(d1, size=c2.shape[2:], mode='bilinear', align_corners=True)
             
        att2 = self.att2(c2)
        d1 = torch.cat([att2, d1], dim=1)
        d1 = self.relu1(self.dec_conv1(d1))
        
        d2 = self.up2(d1)
        if d2.size() != c1.size():
             d2 = F.interpolate(d2, size=c1.shape[2:], mode='bilinear', align_corners=True)

        att3 = self.att3(c1)
        d2 = torch.cat([att3, d2], dim=1)
        d2 = self.relu2(self.dec_conv2(d2))
        
        d3 = self.up3(d2)
        if d3.size() != x.size():
             d3 = F.interpolate(d3, size=x.shape[2:], mode='bilinear', align_corners=True)
             
        d3 = self.relu3(self.dec_conv3(d3))
        
        out = self.final(d3)
        return out

if __name__ == "__main__":
    x = torch.randn((2, 3, 560, 560))
    f = build_unet()
    y = f(x)
    print(y.shape)

torch.Size([2, 1, 560, 560])


In [3]:
# Data
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset

class DriveDataset(Dataset):
    def __init__(self, images_path, masks_path):

        self.images_path = images_path
        self.masks_path = masks_path
        self.n_samples = len(images_path)

    def __getitem__(self, index):
        """ Reading image """
        image = cv2.imread(self.images_path[index], cv2.IMREAD_COLOR)
        image = image/255.0 ## (512, 512, 3)
        image = np.transpose(image, (2, 0, 1))  ## (3, 512, 512)
        image = image.astype(np.float32)
        image = torch.from_numpy(image)

        """ Reading mask """
        mask = cv2.imread(self.masks_path[index], cv2.IMREAD_GRAYSCALE)
        mask = mask/255.0   ## (512, 512)
        mask = np.expand_dims(mask, axis=0) ## (1, 512, 512)
        mask = mask.astype(np.float32)
        mask = torch.from_numpy(mask)
        return image, mask

    def __len__(self):
        return self.n_samples

In [4]:
# Loss
import torch
import torch.nn as nn
import torch.nn.functional as F

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)

        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):

        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = torch.sigmoid(inputs)

        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)

        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss

        return Dice_BCE

In [5]:
# Utils
import os
import time
import random
import numpy as np
import cv2
import torch

""" Seeding the randomness. """
def seeding(seed):
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

""" Create a directory. """
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

""" Calculate the time taken """
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [6]:
# Train
import os
import time
from glob import glob
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader
import torch.nn as nn

# from data import DriveDataset
# from model import build_unet
# from loss import DiceLoss, DiceBCELoss
# from utils import seeding, create_dir, epoch_time

'''训练深度学习模型'''
def train(model, loader, optimizer, loss_fn, device, show_images=False):
    epoch_loss = 0.0

    model.train()
    for i, (x, y) in enumerate(loader):
        x = x.to(device, dtype=torch.float32)
        y = y.to(device, dtype=torch.float32)

        # if i == 1 and show_images:
        #    # 显示第一批图像和掩码
        #     img = x[0].cpu().numpy()  # 假设图像是CHW格式
        #     img = np.transpose(img, (1, 2, 0))  # 转换为HWC格式
        #     img = img[..., ::-1]  # 将BGR转换为RGB
        #     mask = y[0].cpu().numpy()  # 假设掩码是CHW格式
        #     mask = np.transpose(mask, (1, 2, 0))  # 转换为HWC格式

        #     plt.figure(figsize=(12, 6))

        #     plt.subplot(1, 2, 1)
        #     plt.imshow(img)
        #     plt.title("Sample Image")

        #     plt.subplot(1, 2, 2)
        #     plt.imshow(mask, cmap='gray')
        #     plt.title("Corresponding Mask")

        #     plt.show()

        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    #计算整个epoch的平均损失
    epoch_loss = epoch_loss/len(loader)
    return epoch_loss

def evaluate(model, loader, loss_fn, device):
    epoch_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device, dtype=torch.float32)
            y = y.to(device, dtype=torch.float32)

            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            epoch_loss += loss.item()

        epoch_loss = epoch_loss/len(loader)
    return epoch_loss

if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Directories """
    create_dir("files")

    """ Load dataset """
    train_x = sorted(glob("working/new_data/train/image/*"))
    train_y = sorted(glob("working/new_data/train/mask/*"))

    valid_x = sorted(glob("working/new_data/test/image/*"))
    valid_y = sorted(glob("working/new_data/test/mask/*"))

    data_str = f"Dataset Size:\nTrain: {len(train_x)} - Valid: {len(valid_x)}\n"
    print(data_str)

    """ Hyperparameters """
    H = 560
    W = 560
    size = (H, W)
    batch_size = 64
    num_epochs = 100   
    lr = 1e-3
    checkpoint_path = "working/files/drive_checkpoint_ori.pth"

    """ Dataset and loader """
    train_dataset = DriveDataset(train_x, train_y)
    valid_dataset = DriveDataset(valid_x, valid_y)

    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2
    )

    valid_loader = DataLoader(
        dataset=valid_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=2
    )

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')   ## GTX 1060 6GB
    model = build_unet()
    model = model.to(device)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
    loss_fn = DiceBCELoss()

    """ Training the model """
    best_valid_loss = float("inf")

    for epoch in range(num_epochs):
        start_time = time.time()

        #该轮训练的平均损失值 train_loss
        train_loss = train(model, train_loader, optimizer, loss_fn, device, show_images=True)
        #返回验证损失 valid_loss
        valid_loss = evaluate(model, valid_loader, loss_fn, device)

        """ Saving the model """
        if valid_loss < best_valid_loss:
            data_str = f"Valid loss improved from {best_valid_loss:2.4f} to {valid_loss:2.4f}. Saving checkpoint: {checkpoint_path}"
            print(data_str)

            best_valid_loss = valid_loss
            torch.save(model.state_dict(), checkpoint_path)

        end_time = time.time()
        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        data_str = f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s\n'
        data_str += f'\tTrain Loss: {train_loss:.3f}\n'
        data_str += f'\t Val. Loss: {valid_loss:.3f}\n'
        print(data_str)

Dataset Size:
Train: 240 - Valid: 20



Valid loss improved from inf to 1.5080. Saving checkpoint: working/files/drive_checkpoint_ori.pth
Epoch: 01 | Epoch Time: 0m 4s
	Train Loss: 1.518
	 Val. Loss: 1.508

Valid loss improved from 1.5080 to 1.3882. Saving checkpoint: working/files/drive_checkpoint_ori.pth
Epoch: 02 | Epoch Time: 0m 3s
	Train Loss: 1.458
	 Val. Loss: 1.388

Valid loss improved from 1.3882 to 1.2473. Saving checkpoint: working/files/drive_checkpoint_ori.pth
Epoch: 03 | Epoch Time: 0m 3s
	Train Loss: 1.240
	 Val. Loss: 1.247

Epoch: 04 | Epoch Time: 0m 3s
	Train Loss: 1.120
	 Val. Loss: 1.250

Epoch: 05 | Epoch Time: 0m 3s
	Train Loss: 1.092
	 Val. Loss: 1.494

Epoch: 06 | Epoch Time: 0m 3s
	Train Loss: 1.067
	 Val. Loss: 1.578

Epoch: 07 | Epoch Time: 0m 3s
	Train Loss: 1.051
	 Val. Loss: 1.500

Epoch: 08 | Epoch Time: 0m 3s
	Train Loss: 1.032
	 Val. Loss: 1.348

Epoch: 09 | Epoch Time: 0m 3s
	Train Loss: 1.013
	 Val. Loss: 1.277

Epoch: 10 | Epoch Time: 0m 3s
	Train Loss: 0.991
	 Val. Loss: 1.298

Epoch: 11 

In [ ]:
# Test
import os, time
from operator import add
import numpy as np
from glob import glob
import cv2
from tqdm import tqdm
import imageio
import torch
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score, confusion_matrix

# from model import build_unet
# from utils import create_dir, seeding

def calculate_metrics(y_true, y_pred):
    """ Ground truth """
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction """
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    # 使用混淆矩阵计算 TP, FP, FN, TN
    # labels=[0, 1] 确保即使数据中缺少某一类也能返回 2x2 矩阵
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()

    # Dice Coefficient (F1 Score) = 2 * TP / (2 * TP + FP + FN)
    score_f1 = 2 * tp / (2 * tp + fp + fn + 1e-6)

    # Jaccard (IoU) = TP / (TP + FP + FN)
    score_jaccard = tp / (tp + fp + fn + 1e-6)

    # Sensitivity (Recall) = TP / (TP + FN)
    score_recall = tp / (tp + fn + 1e-6)

    # Specificity = TN / (TN + FP)
    score_specificity = tn / (tn + fp + 1e-6)

    # Precision = TP / (TP + FP)
    score_precision = tp / (tp + fp + 1e-6)

    # Accuracy = (TP + TN) / (TP + TN + FP + FN)
    score_acc = (tp + tn) / (tp + tn + fp + fn + 1e-6)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc, score_specificity]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    return mask

if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Folders """
    create_dir("working/results")

    """ Load dataset """
    test_x = sorted(glob("working/new_data/test/image/*"))
    test_y = sorted(glob("working/new_data/test/mask/*"))

    """ Hyperparameters """
    H = 560
    W = 560
    size = (W, H)
    checkpoint_path = "working/files/drive_checkpoint_ori.pth"

    """ Load the checkpoint """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = build_unet()
    model = model.to(device)
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()

    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    time_taken = []
    
    # Lists to store metrics for each image for plotting
    image_indices = []
    jaccard_scores = []
    f1_scores = []
    recall_scores = []
    specificity_scores = []

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        """ Extract the name """
        name = x.split("/")[-1].split(".")[0]

        """ Reading image """
        image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
        ## image = cv2.resize(image, size)
        x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
        x = x/255.0
        x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)

        """ Reading mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
        ## mask = cv2.resize(mask, size)
        y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
        y = y/255.0
        y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
        y = y.astype(np.float32)
        y = torch.from_numpy(y)
        y = y.to(device)

        with torch.no_grad():
            """ Prediction and Calculating FPS """
            start_time = time.time()
            pred_y = model(x)
            pred_y = torch.sigmoid(pred_y)
            total_time = time.time() - start_time
            time_taken.append(total_time)


            score = calculate_metrics(y, pred_y)
            print(f":-- jaccard:{score[0]:1.4f}, f1:{score[1]:1.4f},recall:{score[2]:1.4f},precision:{score[3]:1.3f},acc:{score[4]:1.4f},specificity:{score[5]:1.3f}")
            
            metrics_score = list(map(add, metrics_score, score))
            
            # Store for plotting
            image_indices.append(i)
            jaccard_scores.append(score[0])
            f1_scores.append(score[1])
            recall_scores.append(score[2])
            specificity_scores.append(score[5])
            
            pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
            pred_y = pred_y > 0.5
            pred_y = np.array(pred_y, dtype=np.uint8)

        """ Saving masks """
        ori_mask = mask_parse(mask)
        pred_y = mask_parse(pred_y)
        line = np.ones((size[1], 10, 3)) * 128

        cat_images = np.concatenate(
            [image, line, ori_mask, line, pred_y * 255], axis=1
        )
        cv2.imwrite(f"working/results/{name}.png", cat_images)

    jaccard = metrics_score[0]/len(test_x)
    f1 = metrics_score[1]/len(test_x)
    recall = metrics_score[2]/len(test_x)
    precision = metrics_score[3]/len(test_x)
    acc = metrics_score[4]/len(test_x)
    spec = metrics_score[5]/len(test_x)
    print(f"\nOverall---Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Sensitivity: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f} - Specificity:{spec:1.4f}")

    fps = 1/np.mean(time_taken)
    print("FPS: ", fps)
    


/tmp/ipykernel_1006381/705512160.py:80: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(checkpoint_path, map_location=device))
 15%|█▌        

:-- jaccard:0.5623, f1:0.7199,recall:0.7650,precision:0.680,acc:0.9468,specificity:0.965
:-- jaccard:0.4266, f1:0.5981,recall:0.4726,precision:0.814,acc:0.9349,specificity:0.988
:-- jaccard:0.5486, f1:0.7085,recall:0.6604,precision:0.764,acc:0.9459,specificity:0.977
:-- jaccard:0.4865, f1:0.6546,recall:0.5672,precision:0.774,acc:0.9448,specificity:0.983


 35%|███▌      | 7/20 [00:00<00:00, 18.07it/s]

:-- jaccard:0.5627, f1:0.7201,recall:0.6718,precision:0.776,acc:0.9512,specificity:0.980
:-- jaccard:0.5556, f1:0.7144,recall:0.6465,precision:0.798,acc:0.9496,specificity:0.982
:-- jaccard:0.5380, f1:0.6996,recall:0.6439,precision:0.766,acc:0.9494,specificity:0.980
:-- jaccard:0.5117, f1:0.6770,recall:0.6160,precision:0.751,acc:0.9494,specificity:0.981


 55%|█████▌    | 11/20 [00:00<00:00, 18.65it/s]

...

 75%|███████▌  | 15/20 [00:00<00:00, 18.91it/s]

:-- jaccard:0.5557, f1:0.7144,recall:0.6791,precision:0.754,acc:0.9469,specificity:0.976
:-- jaccard:0.5607, f1:0.7185,recall:0.7055,precision:0.732,acc:0.9554,specificity:0.977
:-- jaccard:0.5242, f1:0.6878,recall:0.7763,precision:0.617,acc:0.9494,specificity:0.963
:-- jaccard:0.5764, f1:0.7313,recall:0.7008,precision:0.765,acc:0.9535,specificity:0.979


100%|██████████| 20/20 [00:01<00:00, 18.27it/s]

:-- jaccard:0.4774, f1:0.6463,recall:0.5633,precision:0.758,acc:0.9479,specificity:0.983
:-- jaccard:0.5311, f1:0.6937,recall:0.6466,precision:0.748,acc:0.9548,specificity:0.981
:-- jaccard:0.5886, f1:0.7410,recall:0.7864,precision:0.701,acc:0.9546,specificity:0.970
:-- jaccard:0.5701, f1:0.7262,recall:0.7635,precision:0.692,acc:0.9576,specificity:0.973

Overall---Jaccard: 0.5275 - F1: 0.6894 - Sensitivity: 0.6539 - Precision: 0.7408 - Acc: 0.9491 - Specificity:0.9778
FPS:  316.1170316998538


: 